**Set environment**

In [2]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
suppressMessages(suppressWarnings(source("../config/config_func.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


## Import data

In [28]:
### set file paths
fdiry  = file.path(
    FD_RES, 
    "results",
    "region",
    "KS91_K562_ASTARRseq_peak_macs_input",
    "summary"
)
fname = paste("peak", "screened", "fcc", "total", "tsv", sep = ".")
fpath = file.path(fdiry, fname)

### read table
dat = read_tsv(fpath, show_col_types = FALSE)

###
dat_peak_screened = dat
print(table(dat$Assay_Type))
print(dim(dat))
head(dat)


               ASTARR        CRISPRi-Growth         CRISPRi-HCRFF 
               246850                 80288                  1330 
 ENCODE-E2G_Benchmark ENCODE-E2G_Prediction                 LMPRA 
                 3007                 84129                 68420 
                TMPRA                WSTARR 
                 1722                246688 
[1] 732434      7


Chrom,Start,End,Peak,Score,Assay_Type,Assay_Label
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
chr1,100006256,100006880,chr1:100006256-100006880,-0.01311792,ASTARR,ASTARR_T
chr1,100010437,100010915,chr1:100010437-100010915,-0.80687947,ASTARR,ASTARR_T
chr1,10002087,10003910,chr1:10002087-10003910,-0.34892859,ASTARR,ASTARR_T
chr1,100021298,100021629,chr1:100021298-100021629,-1.03799571,ASTARR,ASTARR_T
chr1,100023727,100023976,chr1:100023727-100023976,-0.19100207,ASTARR,ASTARR_T
chr1,100027983,100029702,chr1:100027983-100029702,-0.18001582,ASTARR,ASTARR_T


In [6]:
dat = dat_peak_screened
vec = c("ASTARR", "WSTARR", "TMPRA", "LMPRA")
dat = dat %>% dplyr::filter(Assay_Type %in% vec) %>% dplyr::mutate(Assay = Assay_Type)

dat_peak_screened_starrmpra = dat
print(dim(dat))
head(dat)

[1] 563680      8


Chrom,Start,End,Peak,Score,Assay_Type,Assay_Label,Assay
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
chr1,100006256,100006880,chr1:100006256-100006880,-0.01311792,ASTARR,ASTARR_T,ASTARR
chr1,100010437,100010915,chr1:100010437-100010915,-0.80687947,ASTARR,ASTARR_T,ASTARR
chr1,10002087,10003910,chr1:10002087-10003910,-0.34892859,ASTARR,ASTARR_T,ASTARR
chr1,100021298,100021629,chr1:100021298-100021629,-1.03799571,ASTARR,ASTARR_T,ASTARR
chr1,100023727,100023976,chr1:100023727-100023976,-0.19100207,ASTARR,ASTARR_T,ASTARR
chr1,100027983,100029702,chr1:100027983-100029702,-0.18001582,ASTARR,ASTARR_T,ASTARR


## Arrange: convert data from tidy to wide

In [18]:
dat = dat_peak_screened_starrmpra
dat = dat %>%
    dplyr::select(Assay, Peak, Score) %>%
    tidyr::spread(Assay, Score) %>%
    dplyr::select(Peak, ASTARR, WSTARR, TMPRA, LMPRA) %>%
    na.omit

### assign and show
dat_peak_score = dat
print(dim(dat))
head(dat)

[1] 1241    5


Peak,ASTARR,WSTARR,TMPRA,LMPRA
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr11:32870601-32871324,-0.56726545,-0.45442671,-0.7592992,-0.3215166
chr11:32874343-32875070,-0.54920178,-0.23994680,-0.4159155,-0.8170978
chr11:32884749-32885822,-0.66011484,-0.17415901,0.2420861,1.3800391
chr11:32901572-32902485,-0.38438377,-0.09841904,-0.8541150,-0.1454310
chr11:32903491-32904506,-0.05446687,-0.68078185,-0.7215339,1.1180286
chr11:32910855-32912010,0.17567189,-0.01956657,2.3515025,0.1459271


## Rank normalization

In [19]:
fun_ranknorm = function(x){
    return(rank(x)/length(x))
}

In [25]:
dat = dat_peak_score
dat = dat %>% 
    dplyr::mutate(across(2:5, fun_ranknorm)) %>% 
    dplyr::rowwise() %>%
    dplyr::mutate(Mean = mean(c(ASTARR, WSTARR, TMPRA, LMPRA))) %>%
    dplyr::ungroup()

dat_peak_ranknorm = dat
head(dat)

Peak,ASTARR,WSTARR,TMPRA,LMPRA,Mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr11:32870601-32871324,0.12006446,0.06204674,0.1958098,0.22804190,0.1514907
chr11:32874343-32875070,0.13053989,0.21434327,0.3231265,0.08098308,0.1872482
chr11:32884749-32885822,0.08541499,0.26349718,0.5439162,0.78565673,0.4196213
chr11:32901572-32902485,0.22320709,0.35455278,0.1571313,0.31748590,0.2630943
chr11:32903491-32904506,0.52377115,0.01531023,0.2095085,0.74053183,0.3722804
chr11:32910855-32912010,0.68573731,0.44641418,0.9129734,0.43755036,0.6206688


## Save table

In [26]:
### set file directories
fdiry = file.path(FD_RES, "results", "comparison")
fname = "result.score.starrmpra.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_score
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/comparison/result.score.starrmpra.tsv"


In [27]:
### set file directories
fdiry = file.path(FD_RES, "results", "comparison")
fname = "result.ranknorm.starrmpra.overlap.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

dat = dat_peak_ranknorm
write_tsv(dat, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/comparison/result.ranknorm.starrmpra.overlap.tsv"
